In [36]:
import warnings

import joblib
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
%matplotlib inline
warnings.filterwarnings('ignore')

In [37]:
#load your data into the dataframe
my_data = read_csv('crypto_historical_data1.csv')

2. ETHEREUM

In [38]:
eth=my_data[my_data['symbol']=='ETH']#Extracting the ethereum data from the dataframe
eth

,Date,High,Low,Open,Close,Volume,Adj Close,symbol,name
2689,2017-11-09,329.451996,307.056000,308.644989,320.884003,8.932500e+08,320.884003,ETH,Ethereum
2690,2017-11-10,324.717987,294.541992,320.670990,299.252991,8.859860e+08,299.252991,ETH,Ethereum
2691,2017-11-11,319.453003,298.191986,298.585999,314.681000,8.423010e+08,314.681000,ETH,Ethereum
2692,2017-11-12,319.153015,298.513000,314.690002,307.907990,1.613480e+09,307.907990,ETH,Ethereum
2693,2017-11-13,328.415009,307.024994,307.024994,316.716003,1.041890e+09,316.716003,ETH,Ethereum
...,...,...,...,...,...,...,...,...,...
4330,2022-05-08,2638.830566,2498.430176,2636.121826,2517.459961,2.080227e+10,2517.459961,ETH,Ethereum
4331,2022-05-09,2528.258057,2238.062744,2518.508301,2245.430420,3.633357e+10,2245.430420,ETH,Ethereum
4332,2022-05-10,2450.760254,2206.756836,2242.650391,2343.510986,3.820210e+10,2343.510986,ETH,Ethereum
4333,2022-05-11,2441.079346,2018.855591,2342.754150,2072.108643,4.574340e+10,2072.108643,ETH,Ethereum


In [39]:
eth = eth.set_index('Date')

In [40]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                              starts_p=1, starts_q=1,
                              max_p =10, max_q=10,
                              test='adf', seasonal=True,
                              trace= True)
    return auto

arimamodel(eth['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=19081.448, Time=6.10 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=19100.762, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=19095.100, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=19095.273, Time=0.26 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=19099.032, Time=0.04 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=19094.998, Time=2.34 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=19097.554, Time=4.03 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=19083.341, Time=8.99 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=19063.592, Time=9.97 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=19095.663, Time=2.62 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=19077.395, Time=9.23 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : AIC=19065.645, Time=14.15 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=19080.243, Time=2.55 sec
 ARIMA(3,1,4)(0,0,0)[0] intercept   : AIC=19072.710, Time=10.52 sec
 ARIMA(2,1,3)(0,0

ARIMA(order=(2, 1, 3), scoring_args={}, suppress_warnings=True)

In [74]:
# 2,1,3 ARIMA Model
model = ARIMA(eth['Close'], order=(2,1,3))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1646
Model:                 ARIMA(2, 1, 3)   Log Likelihood               -9539.760
Date:                Thu, 12 May 2022   AIC                          19091.521
Time:                        22:44:36   BIC                          19123.954
Sample:                    11-09-2017   HQIC                         19103.547
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.2302      0.081    -15.274      0.000      -1.388      -1.072
ar.L2         -0.6698      0.076     -8.861      0.000      -0.818      -0.522
ma.L1          1.1772      0.083     14.252      0.0

In [75]:
#train test split
n=int(len(eth['Close'])*0.8)
train = eth['Close'][:n]
test = eth['Close'][n:]
print(len(train))
print(len(test))

1316
330


In [76]:
start=len(train)
end=len(train)+len(test)-1
predictions=result.predict(start=start,end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-06-17    2381.987018
2021-06-18    2378.711896
2021-06-19    2244.456637
2021-06-20    2178.260270
2021-06-21    2255.126367
                 ...     
2022-05-08    2649.089385
2022-05-09    2515.212350
2022-05-10    2273.934395
2022-05-11    2346.910810
2022-05-12    2081.129247
Freq: D, Name: predicted_mean, Length: 330, dtype: float64

In [77]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 =  r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 101.99
Mean squared error:, 17654.17
Root mean squared error:, 132.87
r2 score:0.97
Mean absolute percentage error:, 0.03


In [78]:
#saving model to disk
final_model = 'eth_model.sav'
joblib.dump(model, final_model)

['eth_model.sav']

In [79]:
forecast =result.forecast(steps=7,type='level')
forecast

2022-05-13    2023.020424
2022-05-14    2034.141335
2022-05-15    2018.429697
2022-05-16    2030.308893
2022-05-17    2026.219107
2022-05-18    2023.293570
2022-05-19    2029.631804
Freq: D, Name: predicted_mean, dtype: float64

TETHER

In [50]:
usdt = my_data[my_data['symbol'] == 'USDT']  #Extracting the tether data from the dataframe
usdt = usdt.set_index('Date')
usdt

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2017-11-09,1.013270,0.996515,1.010870,1.008180,3.581880e+08,1.008180,USDT,Tether
2017-11-10,1.024230,0.995486,1.006500,1.006010,7.564460e+08,1.006010,USDT,Tether
2017-11-11,1.026210,0.995799,1.005980,1.008990,7.462280e+08,1.008990,USDT,Tether
2017-11-12,1.105910,0.967601,1.006020,1.012470,1.466060e+09,1.012470,USDT,Tether
2017-11-13,1.029290,0.975103,1.004480,1.009350,7.678840e+08,1.009350,USDT,Tether
...,...,...,...,...,...,...,...,...
2022-05-08,1.000063,0.999740,0.999919,0.999854,7.790016e+10,0.999854,USDT,Tether
2022-05-09,1.000134,0.999770,0.999870,0.999909,1.207103e+11,0.999909,USDT,Tether
2022-05-10,1.000134,0.999709,0.999945,0.999791,1.286921e+11,0.999791,USDT,Tether


In [52]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(usdt['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-13319.705, Time=5.04 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-12139.034, Time=0.38 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-13151.091, Time=0.26 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-12713.769, Time=1.61 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=4679.246, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-13293.268, Time=2.87 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-13308.125, Time=14.68 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=-13315.303, Time=4.68 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-13298.006, Time=3.29 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-13277.808, Time=3.55 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-13293.754, Time=3.06 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-13290.057, Time=6.54 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=-13328.089, Time=2.35 sec
 ARIMA(4,0,3)(0,0,0)[0] intercept   : AIC=-13333.224, Time=4.28 sec
 ARIMA

ARIMA(order=(4, 0, 4), scoring_args={}, suppress_warnings=True)

In [80]:
# 4,0,4 ARIMA Model
model = ARIMA(usdt['Close'], order=(4, 0, 4))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1646
Model:                 ARIMA(4, 0, 4)   Log Likelihood                6686.478
Date:                Thu, 12 May 2022   AIC                         -13352.956
Time:                        22:45:22   BIC                         -13298.895
Sample:                    11-09-2017   HQIC                        -13332.910
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0018      0.001   1070.422      0.000       1.000       1.004
ar.L1         -0.3749      0.142     -2.648      0.008      -0.652      -0.097
ar.L2          0.4718      0.146      3.231      0.0

In [81]:
#train test split
n = int(len(usdt['Close']) * 0.8)
train = usdt['Close'][:n]
test = usdt['Close'][n:]
print(len(train))
print(len(test))

1316
330


In [83]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)


2021-06-17    1.000490
2021-06-18    1.000643
2021-06-19    1.001256
2021-06-20    1.001169
2021-06-21    1.001042
                ...   
2022-05-08    1.000221
2022-05-09    1.000171
2022-05-10    1.000177
2022-05-11    1.000128
2022-05-12    0.998470
Freq: D, Name: predicted_mean, Length: 330, dtype: float64

In [84]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.00
Mean squared error:, 0.00
Root mean squared error:, 0.00
r2 score:0.09
Mean absolute percentage error:, 0.00


In [85]:
#saving model to disk
final_model = 'usdt_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.997596
2022-05-14    0.998271
2022-05-15    0.998506
2022-05-16    0.998459
2022-05-17    0.998695
2022-05-18    0.998876
2022-05-19    0.998959
Freq: D, Name: predicted_mean, dtype: float64

BINANCE COIN

In [58]:
bnb = my_data[my_data['symbol'] == 'BNB']  #Extracting the binance coin data from the dataframe
bnb = bnb.set_index('Date')
bnb

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2017-11-09,2.174230,1.893940,2.053140,1.990770,1.919220e+07,1.990770,BNB,Binance Coin
2017-11-10,2.069470,1.644780,2.007730,1.796840,1.115500e+07,1.796840,BNB,Binance Coin
2017-11-11,1.917750,1.614290,1.786280,1.670470,8.178150e+06,1.670470,BNB,Binance Coin
2017-11-12,1.672800,1.462560,1.668890,1.519690,1.529870e+07,1.519690,BNB,Binance Coin
2017-11-13,1.735020,1.517600,1.526010,1.686620,1.223880e+07,1.686620,BNB,Binance Coin
...,...,...,...,...,...,...,...,...
2022-05-08,366.509583,351.143768,365.733002,355.398743,1.875846e+09,355.398743,BNB,Binance Coin
2022-05-09,358.950714,299.348938,355.377411,299.348938,2.943374e+09,299.348938,BNB,Binance Coin
2022-05-10,332.438843,290.586792,298.820709,319.326965,3.210604e+09,319.326965,BNB,Binance Coin


In [59]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                              starts_p=1, starts_q=1,
                              max_p =10, max_q=10,
                              test='adf', seasonal=True,
                              trace= True)
    return auto

arimamodel(bnb['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=13006.523, Time=5.90 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=13065.856, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=13033.106, Time=0.16 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=13038.636, Time=0.42 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=13064.142, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=13018.638, Time=1.33 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=13022.558, Time=2.71 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=13007.517, Time=7.91 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=13019.238, Time=7.11 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=13026.815, Time=2.19 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=13018.819, Time=3.01 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=13022.465, Time=1.45 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=12992.372, Time=10.96 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=12986.648, Time=9.05 sec
 ARIMA(4,1,2)(0,0,

ARIMA(order=(5, 1, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [86]:
# 5,1,5 ARIMA Model
model = ARIMA(bnb['Close'], order=(5,1,5))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1646
Model:                 ARIMA(5, 1, 5)   Log Likelihood               -6473.769
Date:                Thu, 12 May 2022   AIC                          12969.537
Time:                        22:47:01   BIC                          13028.998
Sample:                    11-09-2017   HQIC                         12991.586
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2885      0.032     -8.938      0.000      -0.352      -0.225
ar.L2         -0.1918      0.020     -9.423      0.000      -0.232      -0.152
ar.L3          0.0125      0.028      0.450      0.6

In [87]:
#train test split
n = int(len(bnb['Close']) * 0.8)
train = bnb['Close'][:n]
test = bnb['Close'][n:]
print(len(train))
print(len(test))

1316
330


In [88]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-06-17    358.363733
2021-06-18    354.226479
2021-06-19    340.245502
2021-06-20    340.857335
2021-06-21    342.120290
                 ...    
2022-05-08    366.791189
2022-05-09    350.294553
2022-05-10    309.537442
2022-05-11    309.440128
2022-05-12    276.108089
Freq: D, Name: predicted_mean, Length: 330, dtype: float64

In [89]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))


Mean absolute error:, 13.43
Mean squared error:, 328.88
Root mean squared error:, 18.14
r2 score:0.96
Mean absolute percentage error:, 0.03


In [90]:
#saving model to disk
final_model = 'bnb_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    278.719969
2022-05-14    285.291066
2022-05-15    278.313237
2022-05-16    289.429142
2022-05-17    283.142408
2022-05-18    280.075486
2022-05-19    281.201423
Freq: D, Name: predicted_mean, dtype: float64

USD COIN

In [91]:
usdc = my_data[my_data['symbol'] == 'USDC']  #Extracting the usd coin data from the dataframe
usdc = usdc.set_index('Date')
usdc

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2018-10-08,1.007680,1.001900,1.004100,1.002210,3.829000e+05,1.002210,USDC,USD Coin
2018-10-09,1.008330,1.000030,1.004270,1.006860,1.088030e+05,1.006860,USDC,USD Coin
2018-10-10,1.019080,1.003970,1.004940,1.009620,7.117830e+05,1.009620,USDC,USD Coin
2018-10-11,1.031230,1.001730,1.009940,1.009780,4.177290e+06,1.009780,USDC,USD Coin
2018-10-12,1.018710,1.001140,1.008610,1.012720,1.322240e+06,1.012720,USDC,USD Coin
...,...,...,...,...,...,...,...,...
2022-05-08,1.000631,0.999548,0.999918,1.000087,5.432001e+09,1.000087,USDC,USD Coin
2022-05-09,1.000682,0.999376,0.999994,0.999595,8.493986e+09,0.999595,USDC,USD Coin
2022-05-10,1.000685,0.999278,0.999576,0.999828,8.611992e+09,0.999828,USDC,USD Coin


In [92]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(usdc['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-11104.705, Time=2.59 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-9628.311, Time=0.34 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-10883.970, Time=0.19 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-10283.474, Time=3.28 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3735.754, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-11066.018, Time=5.17 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-11065.127, Time=10.02 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=-11104.391, Time=2.82 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-11079.916, Time=29.43 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-11062.822, Time=1.44 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-11094.578, Time=13.69 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-11088.555, Time=1.79 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=-11102.658, Time=11.79 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-11075.204, Time=1.55 sec

Bes

ARIMA(order=(2, 0, 2), scoring_args={}, suppress_warnings=True)

In [93]:
# 2,0,2 ARIMA Model
model = ARIMA(usdc['Close'], order=(2,0,2))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1313
Model:                 ARIMA(2, 0, 2)   Log Likelihood                5561.603
Date:                Thu, 12 May 2022   AIC                         -11111.207
Time:                        22:50:14   BIC                         -11080.127
Sample:                    10-08-2018   HQIC                        -11099.551
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0029      0.001    726.356      0.000       1.000       1.006
ar.L1          0.4424      0.145      3.043      0.002       0.157       0.727
ar.L2          0.5018      0.139      3.609      0.0

In [94]:
#train test split
n = int(len(usdc['Close']) * 0.8)
train = usdc['Close'][:n]
test = usdc['Close'][n:]
print(len(train))
print(len(test))

1050
263


In [95]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-08-23    1.000352
2021-08-24    1.000464
2021-08-25    1.000443
2021-08-26    1.000374
2021-08-27    1.000398
                ...   
2022-05-08    1.000239
2022-05-09    1.000297
2022-05-10    1.000085
2022-05-11    1.000132
2022-05-12    1.000499
Freq: D, Name: predicted_mean, Length: 263, dtype: float64

In [96]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.00
Mean squared error:, 0.00
Root mean squared error:, 0.00
r2 score:-0.49
Mean absolute percentage error:, 0.00


In [97]:
#saving model to disk
final_model = 'usdc_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    1.000837
2022-05-14    1.000883
2022-05-15    1.000972
2022-05-16    1.001034
2022-05-17    1.001106
2022-05-18    1.001169
2022-05-19    1.001233
Freq: D, Name: predicted_mean, dtype: float64

XRP

In [98]:
xrp = my_data[my_data['symbol'] == 'XRP']  #Extracting the usd coin data from the dataframe
xrp = xrp.set_index('Date')
xrp

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2017-11-09,0.221791,0.214866,0.217911,0.217488,1.479170e+08,0.217488,XRP,XRP
2017-11-10,0.219068,0.205260,0.218256,0.206483,1.410330e+08,0.206483,XRP,XRP
2017-11-11,0.214456,0.205459,0.205948,0.210430,1.345030e+08,0.210430,XRP,XRP
2017-11-12,0.210214,0.195389,0.210214,0.197339,2.511750e+08,0.197339,XRP,XRP
2017-11-13,0.204081,0.197456,0.197472,0.203442,1.325670e+08,0.203442,XRP,XRP
...,...,...,...,...,...,...,...,...
2022-05-08,0.582634,0.562757,0.582399,0.566527,1.505054e+09,0.566527,XRP,XRP
2022-05-09,0.575994,0.480722,0.566519,0.492940,3.297478e+09,0.492940,XRP,XRP
2022-05-10,0.535299,0.478110,0.492440,0.513626,3.455777e+09,0.513626,XRP,XRP


In [99]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(xrp['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-4575.788, Time=6.50 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=1532.187, Time=1.29 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-4578.995, Time=1.46 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-443.993, Time=1.19 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3335.989, Time=0.18 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-4578.336, Time=0.75 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-4578.223, Time=1.19 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-4587.946, Time=7.54 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-4576.302, Time=4.41 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-4578.797, Time=1.74 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=-4579.415, Time=2.25 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=-4576.991, Time=11.55 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-4582.205, Time=3.92 sec

Best model:  ARIMA(2,0,1)(0,0,0)[0] intercept
Total fit time: 43.994 seconds


ARIMA(order=(2, 0, 1), scoring_args={}, suppress_warnings=True)

In [100]:
# 2,0,1 ARIMA Model
model = ARIMA(xrp['Close'], order=(2,0,1))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1646
Model:                 ARIMA(2, 0, 1)   Log Likelihood                2298.990
Date:                Thu, 12 May 2022   AIC                          -4587.979
Time:                        22:52:53   BIC                          -4560.949
Sample:                    11-09-2017   HQIC                         -4577.956
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5233      0.183      2.859      0.004       0.165       0.882
ar.L1          0.0450      0.018      2.555      0.011       0.010       0.080
ar.L2          0.9300      0.018     52.650      0.0

In [101]:
#train test split
n = int(len(xrp['Close']) * 0.8)
train = xrp['Close'][:n]
test = xrp['Close'][n:]
print(len(train))
print(len(test))

1316
330


In [102]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-06-17    0.817948
2021-06-18    0.847474
2021-06-19    0.783099
2021-06-20    0.768360
2021-06-21    0.764184
                ...   
2022-05-08    0.580806
2022-05-09    0.566282
2022-05-10    0.490844
2022-05-11    0.516774
2022-05-12    0.408845
Freq: D, Name: predicted_mean, Length: 330, dtype: float64

In [103]:
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.03
Mean squared error:, 0.00
Root mean squared error:, 0.05
r2 score:0.95
Mean absolute percentage error:, 0.04


In [104]:
#saving model to disk
final_model = 'xrp_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.396659
2022-05-14    0.393544
2022-05-15    0.399677
2022-05-16    0.397056
2022-05-17    0.402642
2022-05-18    0.400455
2022-05-19    0.405552
Freq: D, Name: predicted_mean, dtype: float64

SOLANA

In [106]:
sol = my_data[my_data['symbol'] == 'SOL']  #Extracting the usd coin data from the dataframe
sol = sol.set_index('Date')
sol

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2020-04-10,1.313487,0.694187,0.832005,0.951054,8.736428e+07,0.951054,SOL,Solana
2020-04-11,1.049073,0.765020,0.951054,0.776819,4.386244e+07,0.776819,SOL,Solana
2020-04-12,0.956670,0.762426,0.785448,0.882507,3.873690e+07,0.882507,SOL,Solana
2020-04-13,0.891603,0.773976,0.890760,0.777832,1.821128e+07,0.777832,SOL,Solana
2020-04-14,0.796472,0.628169,0.777832,0.661925,1.674761e+07,0.661925,SOL,Solana
...,...,...,...,...,...,...,...,...
2022-05-08,79.283791,74.527184,78.987633,75.220184,1.439701e+09,75.220184,SOL,Solana
2022-05-09,76.983093,63.269653,75.234459,63.269653,2.773999e+09,63.269653,SOL,Solana
2022-05-10,73.759888,61.026585,63.369312,66.766151,3.677013e+09,66.766151,SOL,Solana


In [107]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(sol['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4685.353, Time=2.43 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4680.147, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4680.761, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4680.754, Time=0.16 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4678.250, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4682.743, Time=0.23 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 3.061 seconds


ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [108]:
# 0,1,0 ARIMA Model
model = ARIMA(sol['Close'], order=(0,1,0))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  763
Model:                 ARIMA(0, 1, 0)   Log Likelihood               -2338.125
Date:                Thu, 12 May 2022   AIC                           4678.250
Time:                        22:54:40   BIC                           4682.886
Sample:                    04-10-2020   HQIC                          4680.035
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        27.0837      0.666     40.673      0.000      25.779      28.389
Ljung-Box (L1) (Q):                   1.39   Jarque-Bera (JB):              1428.58
Prob(Q):                              0.24   Pr

In [109]:
#train test split
n = int(len(sol['Close']) * 0.8)
train = sol['Close'][:n]
test = sol['Close'][n:]
print(len(train))
print(len(test))

610
153


In [110]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-12-11    167.954834
2021-12-12    171.949493
2021-12-13    173.431671
2021-12-14    155.213058
2021-12-15    153.341446
                 ...    
2022-05-08     78.983421
2022-05-09     75.220184
2022-05-10     63.269653
2022-05-11     66.766151
2022-05-12     50.210850
Freq: D, Name: predicted_mean, Length: 153, dtype: float64

In [111]:
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 4.82
Mean squared error:, 41.58
Root mean squared error:, 6.45
r2 score:0.96
Mean absolute percentage error:, 0.04


In [112]:
#saving model to disk
final_model = 'sol_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    46.862148
2022-05-14    46.862148
2022-05-15    46.862148
2022-05-16    46.862148
2022-05-17    46.862148
2022-05-18    46.862148
2022-05-19    46.862148
Freq: D, Name: predicted_mean, dtype: float64

TERRA

In [113]:
luna1 = my_data[my_data['symbol'] == 'LUNA1']  #Extracting the usd coin data from the dataframe
luna1 = luna1.set_index('Date')
luna1

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2019-07-26,1.833167,1.261695,1.704222,1.327436,1.208858e+07,1.327436,LUNA1,Terra
2019-07-27,1.402898,1.221938,1.327436,1.310566,6.033446e+06,1.310566,LUNA1,Terra
2019-07-28,1.368595,1.236858,1.310566,1.272989,1.643709e+06,1.272989,LUNA1,Terra
2019-07-29,1.375941,1.175443,1.262270,1.294661,3.966802e+06,1.294661,LUNA1,Terra
2019-07-30,1.374333,1.266484,1.294435,1.294193,1.820705e+06,1.294193,LUNA1,Terra
...,...,...,...,...,...,...,...,...
2022-05-08,68.271729,59.578342,68.252228,64.081123,5.108105e+09,64.081123,LUNA1,Terra
2022-05-09,65.135132,32.002026,64.126053,32.002026,6.306976e+09,32.002026,LUNA1,Terra
2022-05-10,39.473122,14.148025,31.984324,17.520525,1.045268e+10,17.520525,LUNA1,Terra


In [114]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(luna1['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4853.577, Time=2.39 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4885.011, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4868.918, Time=0.16 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4871.656, Time=0.13 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4883.011, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=4856.313, Time=0.45 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=4856.607, Time=4.96 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=4855.683, Time=2.54 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=4856.093, Time=2.71 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4855.850, Time=1.35 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=4854.901, Time=1.83 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=4855.973, Time=2.21 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=7.99 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=4851.578, Time=1.58 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

ARIMA(order=(2, 1, 2), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [115]:
# 2,1,2 ARIMA Model
model = ARIMA(luna1['Close'], order=(2,1,2))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1022
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -2420.789
Date:                Thu, 12 May 2022   AIC                           4851.578
Time:                        22:58:34   BIC                           4876.220
Sample:                    07-26-2019   HQIC                          4860.934
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.4871      0.104     14.250      0.000       1.283       1.692
ar.L2         -0.6950      0.086     -8.077      0.000      -0.864      -0.526
ma.L1         -1.3904      0.101    -13.801      0.0

In [116]:
#train test split
n = int(len(luna1['Close']) * 0.8)
train = luna1['Close'][:n]
test = luna1['Close'][n:]
print(len(train))
print(len(test))

817
205


In [118]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-10-20    37.909721
2021-10-21    43.178796
2021-10-22    41.356583
2021-10-23    44.136073
2021-10-24    43.491723
                ...    
2022-05-08    66.755738
2022-05-09    62.140569
2022-05-10    27.315871
2022-05-11    11.831927
2022-05-12    -4.882803
Freq: D, Name: predicted_mean, Length: 205, dtype: float64

In [119]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 4.00
Mean squared error:, 29.02
Root mean squared error:, 5.39
r2 score:0.94
Mean absolute percentage error:, 1.96


In [120]:
#saving model to disk
final_model = 'luna1_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13   -4.105822
2022-05-14   -6.233007
2022-05-15   -6.533819
2022-05-16   -5.502731
2022-05-17   -3.760293
2022-05-18   -1.885680
2022-05-19   -0.308905
Freq: D, Name: predicted_mean, dtype: float64

CARDANO

In [121]:
ada = my_data[my_data['symbol'] == 'ADA']  #Extracting the usd coin data from the dataframe
ada = ada.set_index('Date')
ada

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2017-11-09,0.035060,0.025006,0.025160,0.032053,1.871620e+07,0.032053,ADA,Cardano
2017-11-10,0.033348,0.026451,0.032219,0.027119,6.766780e+06,0.027119,ADA,Cardano
2017-11-11,0.029659,0.025684,0.026891,0.027437,5.532220e+06,0.027437,ADA,Cardano
2017-11-12,0.027952,0.022591,0.027480,0.023977,7.280250e+06,0.023977,ADA,Cardano
2017-11-13,0.026300,0.023495,0.024364,0.025808,4.419440e+06,0.025808,ADA,Cardano
...,...,...,...,...,...,...,...,...
2022-05-08,0.763869,0.729272,0.762053,0.739563,1.085127e+09,0.739563,ADA,Cardano
2022-05-09,0.753233,0.610088,0.739509,0.610088,2.306123e+09,0.610088,ADA,Cardano
2022-05-10,0.698520,0.590688,0.609972,0.628963,2.372592e+09,0.628963,ADA,Cardano


In [122]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(ada['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-5091.115, Time=5.23 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-5094.816, Time=0.23 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-5101.178, Time=0.40 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-5100.849, Time=0.24 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-5096.767, Time=0.12 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-5099.978, Time=2.91 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-5105.912, Time=2.60 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-5103.918, Time=2.22 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-5098.276, Time=3.01 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-5099.404, Time=1.60 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-5107.859, Time=1.22 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-5102.791, Time=0.50 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-5103.121, Time=0.54 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-5099.783, Time=1.17 sec
 ARIMA(1,1,2)(0,0,0

ARIMA(order=(2, 1, 3), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [123]:
# 2,1,3 ARIMA Model
model = ARIMA(ada['Close'], order=(2,1,3))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1646
Model:                 ARIMA(2, 1, 3)   Log Likelihood                2563.039
Date:                Thu, 12 May 2022   AIC                          -5114.078
Time:                        23:03:57   BIC                          -5081.645
Sample:                    11-09-2017   HQIC                         -5102.052
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.2895      0.056    -22.965      0.000      -1.400      -1.179
ar.L2         -0.7310      0.051    -14.418      0.000      -0.830      -0.632
ma.L1          1.2298      0.056     21.906      0.0

In [124]:
#train test split
n = int(len(ada['Close']) * 0.8)
train = ada['Close'][:n]
test = ada['Close'][n:]
print(len(train))
print(len(test))

1316
330


In [126]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-06-17    1.496667
2021-06-18    1.479059
2021-06-19    1.424348
2021-06-20    1.389890
2021-06-21    1.430137
                ...   
2022-05-08    0.775041
2022-05-09    0.729198
2022-05-10    0.625645
2022-05-11    0.629370
2022-05-12    0.519248
Freq: D, Name: predicted_mean, Length: 330, dtype: float64

In [127]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))


Mean absolute error:, 0.06
Mean squared error:, 0.01
Root mean squared error:, 0.08
r2 score:0.98
Mean absolute percentage error:, 0.04


In [128]:
#saving model to disk
final_model = 'ada_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.496337
2022-05-14    0.502521
2022-05-15    0.495552
2022-05-16    0.500018
2022-05-17    0.499353
2022-05-18    0.496945
2022-05-19    0.500536
Freq: D, Name: predicted_mean, dtype: float64

AVALANCHE

In [129]:
avax = my_data[my_data['symbol'] == 'AVAX']  #Extracting avalanche coin data from the dataframe
avax = avax.set_index('Date')
avax

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2020-07-13,4.915115,4.481499,4.613968,4.896294,2.834000e+04,4.896294,AVAX,Avalanche
2020-07-14,5.229564,4.762068,4.909865,4.986754,3.957000e+03,4.986754,AVAX,Avalanche
2020-09-22,11.463443,4.125380,4.986754,5.234632,2.880988e+08,5.234632,AVAX,Avalanche
2020-09-23,5.329310,3.982604,5.321654,4.118469,1.730912e+08,4.118469,AVAX,Avalanche
2020-09-24,4.751878,3.539887,3.817925,4.566561,9.611096e+07,4.566561,AVAX,Avalanche
...,...,...,...,...,...,...,...,...
2022-05-08,55.275398,50.935349,55.145802,51.521225,8.378908e+08,51.521225,AVAX,Avalanche
2022-05-09,52.924164,41.972042,51.524044,41.972042,1.228351e+09,41.972042,AVAX,Avalanche
2022-05-10,50.088913,40.383198,41.881672,44.560284,1.624449e+09,44.560284,AVAX,Avalanche


In [130]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(avax['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3335.607, Time=5.12 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3338.830, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3340.583, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3340.583, Time=0.14 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3336.910, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3343.574, Time=0.49 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3343.303, Time=0.53 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=3328.304, Time=4.30 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3340.530, Time=0.78 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=3328.394, Time=4.43 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=3330.078, Time=2.97 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=3328.305, Time=4.17 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=3339.316, Time=1.14 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=3332.269, Time=4.65 sec
 ARIMA(3,1,2)(0,0,0)[0]          

ARIMA(order=(3, 1, 2), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [131]:
# 3,1,2 ARIMA Model
model = ARIMA(avax['Close'], order=(3,1,2))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  600
Model:                 ARIMA(3, 1, 2)   Log Likelihood               -1657.196
Date:                Thu, 12 May 2022   AIC                           3326.392
Time:                        23:06:11   BIC                           3352.763
Sample:                             0   HQIC                          3336.658
                                - 600                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.2036      0.050    -24.063      0.000      -1.302      -1.106
ar.L2         -0.9473      0.051    -18.537      0.000      -1.047      -0.847
ar.L3          0.0057      0.036      0.160      0.8

In [132]:
#train test split
n = int(len(avax['Close']) * 0.8)
train = avax['Close'][:n]
test = avax['Close'][n:]
print(len(train))
print(len(test))

480
120


In [133]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

Date
2022-01-13    93.657635
2022-01-14    90.367307
2022-01-15    91.442866
2022-01-16    91.047232
2022-01-17    93.858501
                ...    
2022-05-08    54.881283
2022-05-09    51.292720
2022-05-10    42.617475
2022-05-11    44.352954
2022-05-12    30.449578
Name: predicted_mean, Length: 120, dtype: float64

In [134]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))


Mean absolute error:, 3.39
Mean squared error:, 18.71
Root mean squared error:, 4.33
r2 score:0.88
Mean absolute percentage error:, 0.05


In [135]:
#saving model to disk
final_model = 'avax_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

600    33.446898
601    32.148118
602    32.425200
603    33.329874
604    31.971096
605    32.751084
606    33.104669
Name: predicted_mean, dtype: float64

HEX

In [136]:
hex = my_data[my_data['symbol'] == 'HEX']  #Extracting the hex coin data from the dataframe
hex = hex.set_index('Date')
hex

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2019-12-17,0.000398,0.000123,0.000200,0.000171,9433.0,0.000171,HEX,HEX
2019-12-18,0.000266,0.000168,0.000172,0.000265,50.0,0.000265,HEX,HEX
2019-12-19,0.000267,0.000180,0.000265,0.000256,32.0,0.000256,HEX,HEX
2019-12-20,0.000257,0.000115,0.000256,0.000129,59.0,0.000129,HEX,HEX
2019-12-21,0.000218,0.000128,0.000129,0.000218,258.0,0.000218,HEX,HEX
...,...,...,...,...,...,...,...,...
2022-05-08,0.185865,0.173683,0.184676,0.174394,20309137.0,0.174394,HEX,HEX
2022-05-09,0.174772,0.143105,0.174394,0.145942,33507889.0,0.145942,HEX,HEX
2022-05-10,0.164788,0.143572,0.145942,0.160441,24528517.0,0.160441,HEX,HEX


In [137]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(hex['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-5537.109, Time=6.01 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-5524.464, Time=0.17 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-5528.072, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-5527.502, Time=0.17 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-5526.305, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-5528.152, Time=1.13 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-5527.455, Time=4.09 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-5539.343, Time=4.35 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-5527.627, Time=2.27 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=-5523.894, Time=7.17 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-5523.817, Time=2.59 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-5538.857, Time=3.85 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=-5525.736, Time=1.40 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=-5522.080, Time=4.10 sec
 ARIMA(3,1,2)(0,0,0

ARIMA(order=(3, 1, 2), scoring_args={}, suppress_warnings=True)

In [138]:
# 3,1,2 ARIMA Model
model = ARIMA(hex['Close'], order=(3,1,2))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  878
Model:                 ARIMA(3, 1, 2)   Log Likelihood                2774.031
Date:                Thu, 12 May 2022   AIC                          -5536.062
Time:                        23:10:15   BIC                          -5507.403
Sample:                    12-17-2019   HQIC                         -5525.100
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1968      0.118     -1.664      0.096      -0.429       0.035
ar.L2         -0.5050      0.100     -5.052      0.000      -0.701      -0.309
ar.L3         -0.0326      0.027     -1.209      0.2

In [139]:
#train test split
n = int(len(hex['Close']) * 0.8)
train = hex['Close'][:n]
test = hex['Close'][n:]
print(len(train))
print(len(test))

702
176


In [140]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-11-18    0.200581
2021-11-19    0.219494
2021-11-20    0.205034
2021-11-21    0.209053
2021-11-22    0.192268
                ...   
2022-05-08    0.187638
2022-05-09    0.173487
2022-05-10    0.141156
2022-05-11    0.161159
2022-05-12    0.136703
Freq: D, Name: predicted_mean, Length: 176, dtype: float64

In [141]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.01
Mean squared error:, 0.00
Root mean squared error:, 0.01
r2 score:0.93
Mean absolute percentage error:, 0.05


In [142]:
#saving model to disk
final_model = 'hex_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.117016
2022-05-14    0.116113
2022-05-15    0.119268
2022-05-16    0.119271
2022-05-17    0.117706
2022-05-18    0.117910
2022-05-19    0.118660
Freq: D, Name: predicted_mean, dtype: float64

DOGECOIN

In [143]:
doge = my_data[my_data['symbol'] == 'DOGE']  #Extracting the doge coin data from the dataframe
doge = doge.set_index('Date')
doge

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2017-11-09,0.001415,0.001181,0.001207,0.001415,6.259550e+06,0.001415,DOGE,Dogecoin
2017-11-10,0.001431,0.001125,0.001421,0.001163,4.246520e+06,0.001163,DOGE,Dogecoin
2017-11-11,0.001257,0.001141,0.001146,0.001201,2.231080e+06,0.001201,DOGE,Dogecoin
2017-11-12,0.001210,0.001002,0.001189,0.001038,3.288960e+06,0.001038,DOGE,Dogecoin
2017-11-13,0.001212,0.001019,0.001046,0.001211,2.481270e+06,0.001211,DOGE,Dogecoin
...,...,...,...,...,...,...,...,...
2022-05-08,0.128017,0.123974,0.127527,0.124207,7.289532e+08,0.124207,DOGE,Dogecoin
2022-05-09,0.125136,0.104824,0.124200,0.104824,1.353326e+09,0.104824,DOGE,Dogecoin
2022-05-10,0.117775,0.100881,0.104708,0.108477,1.723066e+09,0.108477,DOGE,Dogecoin


In [144]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(doge['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-9718.318, Time=3.70 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-9667.211, Time=0.77 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-9693.272, Time=1.15 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-9688.413, Time=0.47 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-9669.186, Time=0.20 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-9718.153, Time=4.64 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-9699.455, Time=3.41 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-9756.027, Time=9.98 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-9708.768, Time=1.75 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=-9750.930, Time=10.51 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-9780.245, Time=11.30 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-9763.789, Time=10.18 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=-9778.453, Time=11.82 sec
 ARIMA(3,1,4)(0,0,0)[0] intercept   : AIC=-9780.789, Time=12.87 sec
 ARIMA(2,1,4)(

ARIMA(order=(5, 1, 4), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [145]:
# 5,1,4 ARIMA Model
model = ARIMA(doge['Close'], order=(5,1,4))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1646
Model:                 ARIMA(5, 1, 4)   Log Likelihood                4952.882
Date:                Thu, 12 May 2022   AIC                          -9885.765
Time:                        23:18:57   BIC                          -9831.710
Sample:                    11-09-2017   HQIC                         -9865.720
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1751      0.019      9.009      0.000       0.137       0.213
ar.L2         -0.2747      0.022    -12.676      0.000      -0.317      -0.232
ar.L3          0.2569      0.018     14.385      0.0

In [147]:
#train test split
n = int(len(doge['Close']) * 0.8)
train = doge['Close'][:n]
test = doge['Close'][n:]
print(len(train))
print(len(test))

1316
330


In [148]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-06-17    0.314051
2021-06-18    0.309464
2021-06-19    0.292885
2021-06-20    0.284609
2021-06-21    0.283624
                ...   
2022-05-08    0.127455
2022-05-09    0.120334
2022-05-10    0.108697
2022-05-11    0.107967
2022-05-12    0.085747
Freq: D, Name: predicted_mean, Length: 330, dtype: float64

In [149]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.01
Mean squared error:, 0.00
Root mean squared error:, 0.01
r2 score:0.94
Mean absolute percentage error:, 0.04


In [150]:
#saving model to disk
final_model = 'doge_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.081427
2022-05-14    0.084329
2022-05-15    0.081856
2022-05-16    0.086972
2022-05-17    0.090433
2022-05-18    0.088024
2022-05-19    0.088173
Freq: D, Name: predicted_mean, dtype: float64

POLKADOT

In [151]:
dot = my_data[my_data['symbol'] == 'DOT']  #Extracting the dot coin data from the dataframe
dot = dot.set_index('Date')
dot

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2020-08-20,3.077784,2.692896,2.787857,2.900080,4.881987e+07,2.900080,DOT,Polkadot
2020-08-21,3.090093,2.730919,2.896923,2.875028,4.931314e+07,2.875028,DOT,Polkadot
2020-08-22,4.510613,2.816762,2.878570,4.484690,4.858011e+08,4.484690,DOT,Polkadot
2020-08-23,4.487058,3.716271,4.487058,3.967066,3.207495e+08,3.967066,DOT,Polkadot
2020-08-24,4.783158,3.812670,3.987413,4.602614,4.076902e+08,4.602614,DOT,Polkadot
...,...,...,...,...,...,...,...,...
2022-05-07,14.352659,13.476185,14.313728,13.775654,5.939830e+08,13.775654,DOT,Polkadot
2022-05-08,13.825541,13.087564,13.774422,13.254074,7.586109e+08,13.254074,DOT,Polkadot
2022-05-09,13.464955,10.767036,13.257022,10.767036,1.529550e+09,10.767036,DOT,Polkadot


In [152]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(dot['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2240.237, Time=1.67 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2239.427, Time=0.15 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2238.762, Time=0.32 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2238.912, Time=0.27 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2237.448, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2236.318, Time=0.42 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2238.291, Time=0.53 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2242.288, Time=1.19 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2240.386, Time=0.20 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2240.340, Time=0.12 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2234.342, Time=0.14 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2236.937, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2236.787, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=2236.315, Time=0.26 sec
 ARIMA(1,1,2)(0,0,0)[0]          

ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [153]:
# 1,1,1 ARIMA Model
model = ARIMA(dot['Close'], order=(1,1,1))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  555
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1114.171
Date:                Thu, 12 May 2022   AIC                           2234.342
Time:                        23:21:53   BIC                           2247.294
Sample:                             0   HQIC                          2239.402
                                - 555                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.8247      0.111     -7.442      0.000      -1.042      -0.607
ma.L1          0.7602      0.124      6.127      0.000       0.517       1.003
sigma2         3.2685      0.084     38.809      0.0

In [154]:
#train test split
n = int(len(dot['Close']) * 0.8)
train = dot['Close'][:n]
test = dot['Close'][n:]
print(len(train))
print(len(test))

444
111


In [155]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

Date
2021-11-07    52.014420
2021-11-08    52.256699
2021-11-09    53.295628
2021-11-10    50.985140
2021-11-11    46.966823
                ...    
2022-05-07    14.192937
2022-05-08    13.901652
2022-05-09    13.191911
2022-05-10    10.974621
2022-05-12    11.162553
Name: predicted_mean, Length: 111, dtype: float64

In [156]:
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 1.27
Mean squared error:, 2.70
Root mean squared error:, 1.64
r2 score:0.97
Mean absolute percentage error:, 0.05


In [157]:
#saving model to disk
final_model = 'dot_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

555    9.365362
556    9.120791
557    9.322486
558    9.156151
559    9.293326
560    9.180199
561    9.273494
Name: predicted_mean, dtype: float64

TERRAUSD

In [159]:
ust = my_data[my_data['symbol'] == 'UST']  #Extracting the terrausd coin data from the dataframe
ust = ust.set_index('Date')
ust

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2020-11-25,1.010752,0.993542,1.010488,0.999572,3.758727e+06,0.999572,UST,TerraUSD
2020-11-26,1.022774,0.982691,0.999360,1.000441,1.621037e+07,1.000441,UST,TerraUSD
2020-11-27,1.003605,0.994698,1.000106,0.999515,4.086973e+06,0.999515,UST,TerraUSD
2020-11-28,1.004033,0.995601,0.999476,0.999325,3.721757e+06,0.999325,UST,TerraUSD
2020-11-29,1.001828,0.997059,0.999246,0.999641,8.569530e+05,0.999641,UST,TerraUSD
...,...,...,...,...,...,...,...,...
2022-05-08,0.999651,0.990503,0.996907,0.996401,2.025913e+09,0.996401,UST,TerraUSD
2022-05-09,0.999014,0.793393,0.995427,0.793393,2.809971e+09,0.793393,UST,TerraUSD
2022-05-10,0.948554,0.684081,0.805948,0.799919,4.905955e+09,0.799919,UST,TerraUSD


In [160]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(ust['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=-2859.012, Time=2.31 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=-2383.161, Time=0.12 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=-2524.380, Time=0.21 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=-2665.463, Time=0.33 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=-2800.792, Time=2.29 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=-2857.835, Time=0.53 sec
 ARIMA(3,2,2)(0,0,0)[0]             : AIC=-2856.856, Time=3.34 sec
 ARIMA(2,2,3)(0,0,0)[0]             : AIC=-2857.646, Time=1.22 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=-2718.917, Time=0.49 sec
 ARIMA(1,2,3)(0,0,0)[0]             : AIC=-2813.841, Time=1.47 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=-2857.955, Time=1.16 sec
 ARIMA(3,2,3)(0,0,0)[0]             : AIC=-2854.947, Time=0.31 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=-2858.196, Time=0.70 sec

Best model:  ARIMA(2,2,2)(0,0,0)[0]          
Total fit time: 14.526 seconds


ARIMA(order=(2, 2, 2), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [178]:
# 1,2,2 ARIMA Model
model = ARIMA(dot['Close'], order=(3,1,2))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  555
Model:                 ARIMA(3, 1, 2)   Log Likelihood               -1114.034
Date:                Thu, 12 May 2022   AIC                           2240.069
Time:                        23:30:17   BIC                           2265.972
Sample:                             0   HQIC                          2250.188
                                - 555                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0626      0.793     -0.079      0.937      -1.617       1.492
ar.L2          0.5925      0.724      0.819      0.413      -0.826       2.011
ar.L3         -0.0211      0.047     -0.451      0.6

In [179]:
#train test split
n = int(len(ust['Close']) * 0.8)
train = ust['Close'][:n]
test = ust['Close'][n:]
print(len(train))
print(len(test))

427
107


In [180]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

Date
2021-10-21    44.378212
2021-10-22    42.883942
2021-10-23    43.305177
2021-10-24    44.091837
2021-10-25    42.417122
                ...    
2022-03-08    16.277360
2022-03-09    17.084297
2022-03-14    17.760007
2022-03-15    17.805020
2022-03-16    17.735620
Name: predicted_mean, Length: 107, dtype: float64

In [165]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 31.43
Mean squared error:, 1104.23
Root mean squared error:, 33.23
r2 score:-288548.72
Mean absolute percentage error:, 31.69


In [181]:
#saving model to disk
final_model = 'ust_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

555    9.405157
556    9.191180
557    9.452221
558    9.302008
559    9.470584
560    9.365526
561    9.475147
Name: predicted_mean, dtype: float64

BINANCEUSD

In [182]:
busd = my_data[my_data['symbol'] == 'BUSD']  #Extracting the binanceusd coin data from the dataframe
busd = busd.set_index('Date')
busd

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2019-09-20,1.006965,0.998945,1.001393,1.001464,2.559360e+05,1.001464,BUSD,Binance USD
2019-09-21,1.007052,1.000069,1.001320,1.001984,1.728540e+05,1.001984,BUSD,Binance USD
2019-09-22,1.007011,0.998558,1.002346,1.003114,1.505900e+05,1.003114,BUSD,Binance USD
2019-09-23,1.012758,0.999545,1.003591,1.002431,1.532224e+06,1.002431,BUSD,Binance USD
2019-09-24,1.059820,0.994858,1.002344,1.007416,4.110761e+06,1.007416,BUSD,Binance USD
...,...,...,...,...,...,...,...,...
2022-05-08,1.001930,0.997955,0.999736,1.000291,1.045222e+10,1.000291,BUSD,Binance USD
2022-05-09,1.002361,0.997820,1.000246,1.000854,8.100177e+09,1.000854,BUSD,Binance USD
2022-05-10,1.002232,0.998038,1.000819,0.999719,8.730362e+09,0.999719,BUSD,Binance USD


In [183]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(busd['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-8413.112, Time=23.51 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-8320.002, Time=0.59 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-8390.784, Time=0.86 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-8371.863, Time=1.46 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=2744.978, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-8417.466, Time=3.47 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=-8392.020, Time=3.78 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-8410.263, Time=1.92 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-8419.927, Time=2.61 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=-8417.903, Time=4.49 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-8445.252, Time=4.36 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=-8400.965, Time=12.16 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=-8431.070, Time=1.00 sec
 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=-8426.616, Time=2.87 sec
 ARIMA(3,0,2)(0,0,

ARIMA(order=(2, 0, 3), scoring_args={}, suppress_warnings=True)

In [186]:
# 2,0,3 ARIMA Model
model = ARIMA(dot['Close'], order=(2,0,3))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  555
Model:                 ARIMA(2, 0, 3)   Log Likelihood               -1116.024
Date:                Thu, 12 May 2022   AIC                           2246.049
Time:                        23:34:32   BIC                           2276.281
Sample:                             0   HQIC                          2257.858
                                - 555                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.9203     11.081      1.527      0.127      -4.798      38.639
ar.L1          0.2176      0.123      1.766      0.077      -0.024       0.459
ar.L2          0.7695      0.121      6.334      0.0

In [187]:
#train test split
n = int(len(busd['Close']) * 0.8)
train = busd['Close'][:n]
test = busd['Close'][n:]
print(len(train))
print(len(test))

772
194


In [188]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

772    15.382694
773    15.393882
774    15.404988
775    15.416014
776    15.426960
         ...    
961    16.533585
962    16.536400
963    16.539193
964    16.541966
965    16.544720
Name: predicted_mean, Length: 194, dtype: float64

In [189]:
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 15.10
Mean squared error:, 227.97
Root mean squared error:, 15.10
r2 score:-341743387.98
Mean absolute percentage error:, 15.10


In [190]:
#saving model to disk
final_model = 'busd_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

555    9.458051
556    9.274525
557    9.683992
558    9.462361
559    9.729243
560    9.616751
561    9.797652
Name: predicted_mean, dtype: float64

SHIBA INU

In [191]:
shib = my_data[my_data['symbol'] == 'SHIB']  #Extracting the  shiba inu data from the dataframe
shib = shib.set_index('Date')
shib

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2020-08-01,0.000000,0.000000,0.000000,0.000000,1.604789e+06,0.000000,SHIB,SHIBA INU
2020-08-02,0.000000,0.000000,0.000000,0.000000,1.930980e+05,0.000000,SHIB,SHIBA INU
2020-08-03,0.000000,0.000000,0.000000,0.000000,1.292250e+05,0.000000,SHIB,SHIBA INU
2020-08-04,0.000000,0.000000,0.000000,0.000000,1.971700e+04,0.000000,SHIB,SHIBA INU
2020-08-05,0.000000,0.000000,0.000000,0.000000,2.686900e+04,0.000000,SHIB,SHIBA INU
...,...,...,...,...,...,...,...,...
2022-05-08,0.000019,0.000018,0.000019,0.000018,5.370736e+08,0.000018,SHIB,SHIBA INU
2022-05-09,0.000019,0.000014,0.000018,0.000014,1.391409e+09,0.000014,SHIB,SHIBA INU
2022-05-10,0.000018,0.000014,0.000014,0.000016,1.923105e+09,0.000016,SHIB,SHIBA INU


In [192]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(shib['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-14090.252, Time=4.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-14098.295, Time=0.38 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-14099.632, Time=1.33 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-13903.853, Time=2.60 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-14877.497, Time=0.41 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-14154.836, Time=3.45 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 12.227 seconds


ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [193]:
# 0,1,0 ARIMA Model
model = ARIMA(shib['Close'], order=(0,1,0))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  650
Model:                 ARIMA(0, 1, 0)   Log Likelihood                7439.748
Date:                Thu, 12 May 2022   AIC                         -14877.497
Time:                        23:36:01   BIC                         -14873.021
Sample:                    08-01-2020   HQIC                        -14875.761
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2       8.87e-12    7.5e-11      0.118      0.906   -1.38e-10    1.56e-10
Ljung-Box (L1) (Q):                  13.82   Jarque-Bera (JB):            103507.66
Prob(Q):                              0.00   Pr

In [194]:
#train test split
n = int(len(shib['Close']) * 0.8)
train = shib['Close'][:n]
test = shib['Close'][n:]
print(len(train))
print(len(test))

520
130


In [195]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2022-01-03    0.000034
2022-01-04    0.000033
2022-01-05    0.000032
2022-01-06    0.000030
2022-01-07    0.000031
                ...   
2022-05-08    0.000019
2022-05-09    0.000018
2022-05-10    0.000014
2022-05-11    0.000016
2022-05-12    0.000012
Freq: D, Name: predicted_mean, Length: 130, dtype: float64

In [196]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.00
Mean squared error:, 0.00
Root mean squared error:, 0.00
r2 score:0.85
Mean absolute percentage error:, 0.04


In [197]:
#saving model to disk
final_model = 'shib_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.000012
2022-05-14    0.000012
2022-05-15    0.000012
2022-05-16    0.000012
2022-05-17    0.000012
2022-05-18    0.000012
2022-05-19    0.000012
Freq: D, Name: predicted_mean, dtype: float64

WRAPPED BITCOIN

In [198]:
wbtc = my_data[my_data['symbol'] == 'WBTC']  #Extracting the wrapped bitcoin data from the dataframe
wbtc = wbtc.set_index('Date')
wbtc

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2019-01-30,3502.762939,3477.248047,3494.277588,3496.455322,1.475000e+03,3496.455322,WBTC,Wrapped Bitcoin
2019-01-31,3535.421143,3435.290527,3496.839844,3466.506592,1.928900e+04,3466.506592,WBTC,Wrapped Bitcoin
2019-02-01,3517.770264,3422.825684,3469.393066,3480.772217,1.364000e+04,3480.772217,WBTC,Wrapped Bitcoin
2019-02-02,3546.404297,3459.112793,3476.335205,3542.095459,1.895000e+03,3542.095459,WBTC,Wrapped Bitcoin
2019-02-03,3544.666992,3436.177734,3540.562744,3477.865967,6.525000e+03,3477.865967,WBTC,Wrapped Bitcoin
...,...,...,...,...,...,...,...,...
2022-05-08,35479.750000,33919.375000,35472.101562,34066.445312,3.540921e+08,34066.445312,WBTC,Wrapped Bitcoin
2022-05-09,34167.175781,30400.675781,34052.562500,30400.675781,6.582155e+08,30400.675781,WBTC,Wrapped Bitcoin
2022-05-10,32528.972656,29945.867188,30384.060547,31029.583984,4.884863e+08,31029.583984,WBTC,Wrapped Bitcoin


In [199]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(wbtc['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=20375.220, Time=5.17 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=20370.323, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=20369.863, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=20369.911, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=20368.720, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=20371.837, Time=0.13 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 5.635 seconds


ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [201]:
# 0,1,0 ARIMA Model
model = ARIMA(wbtc['Close'], order=(0,1,0))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1199
Model:                 ARIMA(0, 1, 0)   Log Likelihood              -10183.360
Date:                Thu, 12 May 2022   AIC                          20368.720
Time:                        23:38:46   BIC                          20373.808
Sample:                    01-30-2019   HQIC                         20370.637
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      1.414e+06   2.73e+04     51.805      0.000    1.36e+06    1.47e+06
Ljung-Box (L1) (Q):                   2.46   Jarque-Bera (JB):              2430.32
Prob(Q):                              0.12   Pr

In [202]:
#train test split
n = int(len(wbtc['Close']) * 0.8)
train = wbtc['Close'][:n]
test = wbtc['Close'][n:]
print(len(train))
print(len(test))

959
240


In [203]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-09-15    47070.902344
2021-09-16    48186.667969
2021-09-17    47830.707031
2021-09-18    47228.113281
2021-09-19    48246.542969
                  ...     
2022-05-08    35479.144531
2022-05-09    34066.445312
2022-05-10    30400.675781
2022-05-11    31029.583984
2022-05-12    28874.722656
Freq: D, Name: predicted_mean, Length: 240, dtype: float64

In [204]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 1169.96
Mean squared error:, 2541577.72
Root mean squared error:, 1594.23
r2 score:0.97
Mean absolute percentage error:, 0.03


In [205]:
#saving model to disk
final_model = 'wbtc_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    29422.316406
2022-05-14    29422.316406
2022-05-15    29422.316406
2022-05-16    29422.316406
2022-05-17    29422.316406
2022-05-18    29422.316406
2022-05-19    29422.316406
Freq: D, Name: predicted_mean, dtype: float64

NEAR PROTOCOL

In [206]:
near = my_data[my_data['symbol'] == 'NEAR']  #Extracting the near protocol data from the dataframe
near = near.set_index('Date')
near

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2020-10-14,1.694560,1.064818,1.694560,1.185806,9.469227e+07,1.185806,NEAR,NEAR Protocol
2020-10-15,1.249225,1.088780,1.177929,1.125557,3.493342e+07,1.125557,NEAR,NEAR Protocol
2020-10-16,1.151711,0.814550,1.124601,0.817511,3.702088e+07,0.817511,NEAR,NEAR Protocol
2020-10-17,0.853755,0.727224,0.817511,0.808136,2.363017e+07,0.808136,NEAR,NEAR Protocol
2020-10-18,0.873260,0.802059,0.809037,0.871674,1.437647e+07,0.871674,NEAR,NEAR Protocol
...,...,...,...,...,...,...,...,...
2022-05-08,11.207347,9.941610,10.439566,11.173626,9.316645e+08,11.173626,NEAR,NEAR Protocol
2022-05-09,11.580922,9.536600,11.170822,9.536600,1.603095e+09,9.536600,NEAR,NEAR Protocol
2022-05-10,10.858898,8.935426,9.515079,9.604581,1.605930e+09,9.604581,NEAR,NEAR Protocol


In [207]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(near['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=2.45 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1136.169, Time=0.17 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1137.947, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1137.927, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1134.311, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.75 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 3.649 seconds


ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [208]:
# 0,1,0 ARIMA Model
model = ARIMA(dot['Close'], order=(0,1,0))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  555
Model:                 ARIMA(0, 1, 0)   Log Likelihood               -1117.724
Date:                Thu, 12 May 2022   AIC                           2237.448
Time:                        23:40:08   BIC                           2241.765
Sample:                             0   HQIC                          2239.134
                                - 555                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2         3.3109      0.078     42.405      0.000       3.158       3.464
Ljung-Box (L1) (Q):                   2.67   Jarque-Bera (JB):              2862.91
Prob(Q):                              0.10   Pr

In [210]:
#train test split
n = int(len(sol['Close']) * 0.8)
train = sol['Close'][:n]
test = sol['Close'][n:]
print(len(train))
print(len(test))

610
153


In [211]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

610    9.068801
611    9.068801
612    9.068801
613    9.068801
614    9.068801
         ...   
758    9.068801
759    9.068801
760    9.068801
761    9.068801
762    9.068801
Name: predicted_mean, Length: 153, dtype: float64

In [212]:
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 105.77
Mean squared error:, 12323.16
Root mean squared error:, 111.01
r2 score:-9.84
Mean absolute percentage error:, 0.91


In [213]:
#saving model to disk
final_model = 'near_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

555    9.068801
556    9.068801
557    9.068801
558    9.068801
559    9.068801
560    9.068801
561    9.068801
Name: predicted_mean, dtype: float64

POLYGON

In [214]:
matic = my_data[my_data['symbol'] == 'MATIC']  #Extracting the polygon data from the dataframe
matic= matic.set_index('Date')
matic

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2019-04-28,0.004545,0.004317,0.004521,0.004421,8.408136e+06,0.004421,MATIC,Polygon
2019-04-29,0.004492,0.003471,0.004418,0.003471,8.267044e+06,0.003471,MATIC,Polygon
2019-04-30,0.005078,0.003458,0.003462,0.004478,2.507271e+07,0.004478,MATIC,Polygon
2019-05-01,0.004673,0.004071,0.004440,0.004374,1.056735e+07,0.004374,MATIC,Polygon
2019-05-02,0.004700,0.004096,0.004366,0.004198,6.714908e+06,0.004198,MATIC,Polygon
...,...,...,...,...,...,...,...,...
2022-05-08,1.026281,0.965592,1.021219,0.974345,5.716089e+08,0.974345,MATIC,Polygon
2022-05-09,0.990015,0.799419,0.974406,0.821915,9.927299e+08,0.821915,MATIC,Polygon
2022-05-10,0.982827,0.799712,0.820769,0.890409,1.574312e+09,0.890409,MATIC,Polygon


In [216]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(matic['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-2660.962, Time=4.46 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-2637.413, Time=0.33 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-2650.277, Time=0.25 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-2649.686, Time=0.26 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-2639.350, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-2657.059, Time=3.13 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-2646.615, Time=2.77 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-2658.024, Time=7.34 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-2659.452, Time=4.99 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-2657.518, Time=1.74 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-2658.095, Time=6.15 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-2653.374, Time=5.19 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-2657.474, Time=6.77 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=-2662.920, Time=3.54 sec
 ARIMA(1,1,2)(0,0,0

ARIMA(order=(2, 1, 2), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [217]:
# 2,1,2 ARIMA Model
model = ARIMA(dot['Close'], order=(2,1,2))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                  555
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -1114.132
Date:                Thu, 12 May 2022   AIC                           2238.263
Time:                        23:45:46   BIC                           2259.849
Sample:                             0   HQIC                          2246.696
                                - 555                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0308      1.159     -0.027      0.979      -2.303       2.242
ar.L2          0.6582      0.940      0.700      0.484      -1.185       2.501
ma.L1         -0.0391      1.147     -0.034      0.9

In [218]:
#train test split
n= int(len(matic['Close']) * 0.8)
train = matic['Close'][:n]
test = matic['Close'][n:]
print(len(train))
print(len(test))

888
223


In [219]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

888     9.384372
889     9.384372
890     9.384372
891     9.384372
892     9.384372
          ...   
1106    9.384372
1107    9.384372
1108    9.384372
1109    9.384372
1110    9.384372
Name: predicted_mean, Length: 223, dtype: float64

In [220]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 7.69
Mean squared error:, 59.27
Root mean squared error:, 7.70
r2 score:-374.14
Mean absolute percentage error:, 4.86


In [221]:
#saving model to disk
final_model = 'matic_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

555    9.391631
556    9.176439
557    9.395564
558    9.247165
559    9.395971
560    9.293705
561    9.394804
Name: predicted_mean, dtype: float64

CRYPTO.COM

In [222]:
cro = my_data[my_data['symbol'] == 'CRO']  #Extracting the crypto.com data from the dataframe
cro = cro.set_index('Date')
cro

,High,Low,Open,Close,Volume,Adj Close,symbol,name
Date,,,,,,,,
2018-12-14,0.019970,0.019350,0.019768,0.019606,856990.0,0.019606,CRO,Crypto.com
2018-12-15,0.019845,0.016150,0.019542,0.017243,989710.0,0.017243,CRO,Crypto.com
2018-12-16,0.017496,0.014127,0.017213,0.015301,717174.0,0.015301,CRO,Crypto.com
2018-12-17,0.016723,0.011487,0.015186,0.014448,615623.0,0.014448,CRO,Crypto.com
2018-12-18,0.017215,0.013553,0.014308,0.016260,764474.0,0.016260,CRO,Crypto.com
...,...,...,...,...,...,...,...,...
2022-05-08,0.271765,0.255696,0.271765,0.256433,45159282.0,0.256433,CRO,Crypto.com
2022-05-09,0.257775,0.209882,0.256373,0.209882,97418948.0,0.209882,CRO,Crypto.com
2022-05-10,0.241302,0.205769,0.209508,0.230374,110785498.0,0.230374,CRO,Crypto.com


In [223]:
# getting the best order with autoarima
def arimamodel(series):
    auto = pm.auto_arima(series,
                         starts_p=1, starts_q=1,
                         max_p=8, max_q=8,
                         test='adf', seasonal=True,
                         trace=True)
    return auto

arimamodel(cro['Close'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-7094.637, Time=7.83 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-7028.667, Time=0.19 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-7043.283, Time=0.51 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-7039.349, Time=0.23 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-7030.568, Time=0.09 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-7093.573, Time=4.55 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-7064.739, Time=2.23 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-7080.501, Time=6.60 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-7090.904, Time=6.89 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-7091.954, Time=2.66 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-7092.391, Time=7.71 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-7062.679, Time=1.41 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-7115.023, Time=7.84 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=-7136.843, Time=10.23 sec
 ARIMA(4,1,2)(0,0,

ARIMA(order=(4, 1, 3), scoring_args={}, suppress_warnings=True)

In [224]:
# 4,1,3 ARIMA Model
model = ARIMA(cro['Close'], order=(4,1,3))
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 1246
Model:                 ARIMA(4, 1, 3)   Log Likelihood                3576.268
Date:                Thu, 12 May 2022   AIC                          -7136.535
Time:                        23:49:28   BIC                          -7095.520
Sample:                    12-14-2018   HQIC                         -7121.113
                         - 05-12-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.0370      0.059    -17.537      0.000      -1.153      -0.921
ar.L2         -0.1591      0.090     -1.764      0.078      -0.336       0.018
ar.L3          0.3331      0.044      7.630      0.0

In [225]:
#train test split
n = int(len(cro['Close']) * 0.8)
train = cro['Close'][:n]
test = cro['Close'][n:]
print(len(train))
print(len(test))

996
250


In [226]:
start = len(train)
end = len(train) + len(test) - 1
predictions = result.predict(start=start, end=end, dynamic=False, type='level')
predictions
# df = pd.DataFrame({'Actual': test, 'Predicted': predictions})
# print(df)

2021-09-05    0.173003
2021-09-06    0.183517
2021-09-07    0.206768
2021-09-08    0.172235
2021-09-09    0.169620
                ...   
2022-05-08    0.270137
2022-05-09    0.252896
2022-05-10    0.213964
2022-05-11    0.219802
2022-05-12    0.197535
Freq: D, Name: predicted_mean, Length: 250, dtype: float64

In [227]:
#evaluate
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test, predictions)
#the r2 for the train dataset
mape = mean_absolute_percentage_error(test, predictions)

#printing the model evaluation values
print('Mean absolute error:, {:.2f}'.format(mae))
print('Mean squared error:, {:.2f}'.format(mse))
print('Root mean squared error:, {:.2f}'.format(rmse))
print('r2 score:{:.2f}'.format(r2))
print('Mean absolute percentage error:, {:.2f}'.format(mape))

Mean absolute error:, 0.02
Mean squared error:, 0.00
Root mean squared error:, 0.03
r2 score:0.97
Mean absolute percentage error:, 0.05


In [228]:
#saving model to disk
final_model = 'cro_model.sav'
joblib.dump(model, final_model)
forecast = result.forecast(steps=7, type='level')
forecast

2022-05-13    0.169956
2022-05-14    0.175397
2022-05-15    0.162372
2022-05-16    0.168803
2022-05-17    0.164190
2022-05-18    0.164772
2022-05-19    0.164266
Freq: D, Name: predicted_mean, dtype: float64